# TMA4320 - Våren 2019, Prosjekt 1
## Blind source separation
**Gruppe 4, Mads Adrian Simonsen**

I dette prosjektet har vi tatt utgangspunkt i det såkalte cocktailselskap-problemet. Vi har flere personer som snakker i et rom. Om man prøver å ta opp lyden fra rommet og spiller av, hører vi bare et kaos av alle lydkildene samtidig. Vi ønsker å separere disse ulike lydkildene slik at vi kan høre hver enkelt person, hva de sier. For å få til det, trenger vi minst like mange mikrofoner som antall personer som prater, og disse mikrofonene kreves å være plassert på forskjellige steder i rommet, slik at vi får ulik vektet amplitude fra de forskjellige kildene. Når vi har samlet inn alle opptakene, er jobben vår å lage et program som separerer alle lydkildene fra hverandre, slik at vi kan spille av lydkildene hver for seg.

Algoritmen for programmet kalles for *uavhengig komponentsanalyse*, og tar utgangspunkt i

"A. Hyvärinen and E. Oja, *Independent component analysis: algorithms and applications*, Neural networks 13 (2000) 411–413".

Vi antar at vi har $d$ lydkilder $\mathbf{s}_1(t), ..., \mathbf{s}_d(t)$, og $d$ mikser av lydkildene hvor alle miksene er vektet forskjellig. Vi antar at hver lydkilde er representert i en radvektor med $N$ elementer:
$$
\mathbf{s}_j = [\mathrm{s}_j(t_1), \mathrm{s}_j(t_2), \dots, \mathrm{s}_j(t_N)].
$$

La de registrerte lydmiksene være $\mathbf{x}_i$, som også er en radvektor med $N$ elementer. Resultatet blir en $d \times N$ matrise:
$$\mathbf{x} = 
\left[
\begin{matrix}
    x_{1}(t_1) & x_{1}(t_2) & x_{1}(t_3) & \dots  & x_{1}(t_N) \\
    x_{2}(t_1) & x_{2}(t_2) & x_{2}(t_3) & \dots  & x_{2}(t_N) \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    x_{d}(t_1) & x_{d}(t_2) & x_{d}(t_3) & \dots  & x_{d}(t_N)
\end{matrix}
\right].
$$

Målet vårt å finne en $d \times d$ matrise $\mathbf{W}$ slik at vi får utledet en approksimasjon av de ulike lydkildene $\mathbf{s}$:
$$\mathbf{s}\approx \mathbf{y} = \mathbf{W} \cdot \tilde{\mathbf{x}} = 
\left[
\begin{matrix}
    w_{11} & w_{12} & \dots  & w_{1d} \\
    w_{21} & w_{22} & \dots  & w_{2d} \\
    \vdots & \vdots & \ddots & \vdots \\
    w_{d1} & w_{d2} & \dots  & w_{dd}
\end{matrix}
\right]
\left[
\begin{matrix}
    \tilde{x}_{1}(t_1) & \tilde{x}_{1}(t_2) & \tilde{x}_{1}(t_3) & \dots  & \tilde{x}_{1}(t_N) \\
    \tilde{x}_{2}(t_1) & \tilde{x}_{2}(t_2) & \tilde{x}_{2}(t_3) & \dots  & \tilde{x}_{2}(t_N) \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    \tilde{x}_{d}(t_1) & \tilde{x}_{d}(t_2) & \tilde{x}_{d}(t_3) & \dots  & \tilde{x}_{d}(t_N)
\end{matrix}
\right],
$$

hvor $\mathbf{y}$ er vår approksimasjon av $\mathbf{s}$, $\tilde{\mathbf{x}}$ er en prosessert transformasjon av $\mathbf{x}$, og $\mathbf{W}$ er en matrise som skal maksimere ikke-gaussiskhet. Algoritmen består av **2 preprosesseringsskritt**, og **1 maksimeringsskritt**:

I det første preprosesseringsskrittet, skal vi sentrere radene i $\mathbf{x}$, slik at summen av hver rad blir lik 0, med andre ord er forventningsverdien lik 0, og det blir da litt enklere formler for oss å bruke i algoritmen.

I det andre preprosesseringsskrittet skal vi transformere lydmiksene slik at de blir ukorrelerte med varians 1. Vi har nå matrisen $\tilde{\mathbf{x}}$.

Det siste som skal gjøres er å finne en matrise som maksimerer ikke-gaussiskhet. Vi starter med en tilfeldig $d \times d$ matrise $\mathbf{W}_k$, finner en midlertidig approksimasjon av $\mathbf{s}$, som vi kaller $\mathbf{s}_k$, og anvender en generell funksjon $G(u)$ og dens deriverte $G'(u)$. Det finnes minst to gode valg for $G(u)$, og det er
$$
G_1(u) = 4u^3,\qquad G_2(u) = u\cdot e^{\frac{-u^2}{2}},
$$
hvor subskriptene 1 og 2 svarer til *Kurtose* og *Negentropy* respektivt. Etter $\mathbf{s}_k$ har blitt anvendt av funksjonene, skal den resulterende matrisen optimeres, så ortonormaliseres. Den ortonormert matrisen som returneres kaller vi $\mathbf{W}_{k+1}$, som vi sjekker opp mot $\mathbf{W}_k$. Vi får da et avvik $\delta$. Dersom $\delta$ > vår toleranseverdi, kjøres $\mathbf{W}_{k+1}$ gjennom samme prosess og returnerer $\mathbf{W}_{k+2}$. Denne prosessen fortsetter helt til $\delta$ < vår toleranseverdi. Denne matrisen som tilfredsstiller toleransekravet kaller vi $\mathbf{W}$, og vi fullfører algoritmen ved å kalkulere vår approksimasjon av $\mathbf{s}$, som vi kaller $\mathbf{y}$. Altså:
$$
    \mathbf{y} = \mathbf{W}\cdot \tilde{\mathbf{x}}.
$$

Vi skal teste dette her på først **3 tildelte lydmikser** som har 3 lydkilder, deretter gjør vi det samme med **4 lydfiler** som er hentet fra nettet.

In [1]:
"""KODE HENTET FRA MAL"""
import numpy as np
from wav_file_loader import read_wavefiles

paths = ['audio/mix_1.wav', 'audio/mix_2.wav', 'audio/mix_3.wav']
data, sampling_rate = read_wavefiles(paths)
num_signals = data.shape[0]

In [2]:
"""KODE HENTET FRA MAL"""
def normalize_audio(data, n):
    abs_data = np.absolute(data)
    maximums = np.amax(abs_data,1)
    # Divide each row by a different vector element:
    data = data / maximums.reshape((n,1))
    return data

data = normalize_audio(data, num_signals)

In [3]:
"""KODE HENTET FRA MAL"""
import IPython.display as ipd

ipd.display(ipd.Audio(data[0,:], rate=sampling_rate))
ipd.display(ipd.Audio(data[1,:], rate=sampling_rate))
ipd.display(ipd.Audio(data[2,:], rate=sampling_rate))

## Her starter definisjonene av funksjonene

In [4]:
def center_rows(Z):
    """
    Input: Vår d x N matrise x, som vi her kaller Z
    Output: Forskyvd matrise av Z, Zc med midlere verdi lik null i hver rad
    """
    d = Z.shape[0]          # Antall rader i matrisen
    N = Z.shape[1]          # Antall kolonner
    Zc = np.zeros((d, N))   # Oppretter en ny matrise med samme dimensjoner som Z
    mus = np.zeros(d)
    for i in range(d):
        mus[i] = np.sum(Z[i])/N  # her oppretter vi en liste over gjennomsnittelig verdi i hver rad av Z
        for j in range(N):
            Zc[i][j] = Z[i][j] - mus[i]  # Trekker ifra gj.sn. til hver rad i Z slik at Zc får midlere verdi lik 0
        #TEST: skal gi 0
        #print(np.sum(Zc[i]))
    return Zc


def whiten_rows(Zc):
    """
    Input: Vår forskyvde matrise Zc
    Output: Zc transformert til en ukorrelert matrise med varians 1, som vi kaller Zw
    """
    
    C = np.cov(Zc)       # C er kovariansmatrisen til Zc
    U, S, _ = np.linalg.svd(C, full_matrices=False) # Faktoriserer C til U * np.diag(S) * _
                                                    # hvor U og _ er unitære matriser, og S er en vektor
                                                    # med de singulære verdiene i synkende rekkefølge
    T = U @ np.diag(1 / np.sqrt(S)) @ U.T   # Matrisemultiplikasjon gir den inverse kvadratroten av kovariansmatrisen
                                            # C^(-1/2), U.T er U transponert
    Zw = T @ Zc       # Transformerer Zc til den ukorrelerte matrisen med varians 1
    #TEST: Skal gi 1
    #print(np.var(Zw))
    return Zw

In [5]:
def normalize_rownorms(Wd):
    """
    Input: En matrise Wd
    Output: Wd med normerte rader.
    """
    
    d = Wd.shape[0]
    for i in range(d):
        Wd[i] = Wd[i]/np.linalg.norm(Wd[i]) # Her deler vi hver rad på normen, for å få norm = 1
        #TEST: skal gi 1
        #print(np.linalg.norm(Wd[i]))
    return Wd

In [6]:
def decorrelate_weights(W):
    """
    Input: En d x d matrise W
    Output: W projisert til en ortogonal matrise ved transformasjonen Wd = (WW^T)^{-1/2} W. Wd er output-argumentet.
    """
    
    Q = W @ W.T
    U, S, _ = np.linalg.svd(Q, full_matrices=False) # Faktoriserer (WW^T) til U * np.diag(S) * _ slik som i whiten_rows(Z)
    T = (U @ np.diag(1 / np.sqrt(S)) @ U.T)   # Matrisemultiplikasjon gir (WW^T)^(-1/2)
    Wd = T @ W # Transformerer W til dens ortogonale projeksjon Wd
    #TEST: skal gi identity matrix, fordi T*T = Q^(-1/2)*Q^(-1/2) = Q^(-1)
    #print(Q @ (T@T))
    return Wd

In [7]:
def update_W(W, Zcw, negentropy=False):
    """
    Input: W som er vår d x d matrise W_k og skal maksimeres for ikke-gaussiskhet,
           Zcw som er vår sentrerte og ukorrelerte x, kjent som tilde{x}, og
           negentropy, valgfri boolsk verdi som avgjør vilken G funksjon som anvendes, hvor kurtose er satt som standard
    Output: Vår nye W, altså W_{k+1} som har blitt optimert og ortogonalisert 
    """
    
    # Først velger vi G-funksjonen vår
    if negentropy: # Negentropy
        G = lambda u: u*np.exp(-u**2/2)
        GPrime = lambda u: np.exp(-u**2/2)*(1-u**2)
    else: # Kurtose
        G = lambda u: 4*(u**3)
        GPrime = lambda u: 3*4*(u**2)
    
    # TRINN 1: Optimering
    d = Zcw.shape[0]
    N = Zcw.shape[1]
    s_k = W @ Zcw # Vår foreløpige approksimasjon av lydkildene
    sG = G(s_k)            # kjører s_k gjennom G
    sGPrime = GPrime(s_k)  # og G'
    E = np.zeros(d)
    for i in range(d):
        E[i] = np.mean(sGPrime[i]) # Tildeler vektoren E med de midlere verdiene av radene til G'(s_k)
    WPlus = 1/N*(sG @ Zcw.T) - np.diag(E) @ W # Utfører nødvendig matematikk for optimeringen
    WPlus = normalize_rownorms(WPlus) # Normerer hver rad av vår nye d x d matrise Wplus
    
    # TRINN 2: Ortogonalisering
    W_kPlus1 = decorrelate_weights(WPlus)
    return W_kPlus1

#Test hentet fra blackboard:
#W = np.array([[1,0],[0,1]])
#Zcw = np.array([[1,2,3,4],[2,3,4,5]])
#W_new = update_W(W, Zcw)
#expected = np.array([[-0.1485, 0.9889],[0.9889, 0.1485]])
#assert(np.allclose(W_new, expected, atol=1e-3))

In [8]:
def measure_of_convergence(W1, W2):
    """
    Input: Våre d x d matriser, W_k og den oppdaterte W_{k+1}
    Output: Avviket mellom inputargumentene, betegnet som delta, som senere sjekkes opp mot en toleranseverdi
    """
    d = W1.shape[0]
    deltaArr = np.zeros(d)
    for i in range(d):
        deltaArr[i] = 1-np.abs(np.sum(W2[i] @ W1[i])) # Vi får en vektor med forskjellige avvik mellom 0 og 1
    delta = np.amax(deltaArr) # Vi lar delta være den høyeste verdien i vektoren deltaArr
    return delta

#Test hentet fra blackboard:
#A = np.array([[1,2,3],[4,5,6],[8,9,10]])
#B = np.array([[3,2,1],[5,4,3],[4,3,2]])
#delta = measure_of_convergence(A,B)
#expected = -9
#assert(delta-expected < 1e-2)

In [9]:
import warnings

def fast_ICA(Z, signals_to_find, negentropy=False, tol=1e-10, max_iter=100):
    """ Her implementeres alle funksjonene.
    
    Input: Z er vår uprosesserte x, altså lydmiksene
           signals_to_find: d antall kilder, i vår situasjon er d = 3
           tol er toleransen vi har satt, standardverdi er 1.0e-10
           negentropy, bestemmer om vi bruker negentropy eller kurtose, kurtose er satt som standard
           max_iter er antall iterasjoner vi maksimalt tillater, for å unngå en uendelig løkke
    Output: Z_ica, vår prosesserte x, altså tilde{x}
            W, den konvergerte d x d matrisen
    """
    Zc = center_rows(Z)    # Sentrerer radene til x, og får forventet verdi lik 0
    Z_ica = whiten_rows(Zc)    # ukorrelerer slik at vi får varians 1, vi har nå fått tilde{x}
    W_0 = np.random.rand(signals_to_find, signals_to_find)  # Vi deklarerer en tilfeldig d x d matrise W_0
    W1 = normalize_rownorms(W_0)                            # normerer den
    delta = 1 # Startverdi for å komme inn i løkka
    number_of_iterations = 0
    while delta > tol and number_of_iterations < max_iter:
        # do an iteration to get a new W-iterate
        W2 = update_W(W1, Z_ica, negentropy) # Her får vi vår foreløpige W_{k+1}
        delta = measure_of_convergence(W1, W2) # Sjekker avviket mellom W_k og W_{k+1}
        W1 = W2 # Setter W_k <- W_k+1 så vi er forberdt på neste iterasjon om ikke toleransekravet holder
        number_of_iterations += 1
        print('#', number_of_iterations, ", delta = ", "{:.2e}".format(delta), sep='')
    # Sjekker konvergens og returnerer ønsket verdi dersom konvergens er oppnådd
    if number_of_iterations == max_iter:
        warnings.warn("delta did not converge")
    else:
        return Z_ica, W2   

## Resultat
Under får vi se resultatet, først med kurtose, så med negentropy. Vi skal også se på hvor mye arbeid som allerede er gjort, bare etter de to preprosesseringstrinnene. Vi velger toleranseverdi $1.0\cdot 10^{-15}$, for å oppnå best mulig resultat uten divergens.

### Kurtose

In [10]:
tol = 1.0e-15
kurtosis = False

Z1, W1 = fast_ICA(data, num_signals, kurtosis, tol)
y1 = W1 @ Z1 # Her er den endelige approksimasjonen av lydkildene s med Kurtose

sepData1 = normalize_audio(y1, num_signals) # gir lydkildene omtrent samme lydstyrke

#1, delta = 3.09e-01
#2, delta = 1.20e-01
#3, delta = 2.44e-02
#4, delta = 4.98e-04
#5, delta = 1.56e-05
#6, delta = 5.06e-07
#7, delta = 1.67e-08
#8, delta = 5.55e-10
#9, delta = 1.85e-11
#10, delta = 6.16e-13
#11, delta = 2.02e-14
#12, delta = 7.77e-16


In [11]:
# Her kan du spille av resultatet hvor vi har brukt Kurtose
ipd.display(ipd.Audio(sepData1[0,:], rate=sampling_rate))
ipd.display(ipd.Audio(sepData1[1,:], rate=sampling_rate))
ipd.display(ipd.Audio(sepData1[2,:], rate=sampling_rate))

### Negentropy

In [12]:
negentropy = True

Z2, W2 = fast_ICA(data, num_signals, negentropy, tol)
y2 = W2 @ Z2 # Her er den endelige approksimasjonen av lydkildene s med Negentropy

sepData2 = normalize_audio(y2, num_signals) # gir lydkildene omtrent samme lydstyrke

#1, delta = 3.42e-01
#2, delta = 4.24e-02
#3, delta = 1.43e-04
#4, delta = 2.67e-07
#5, delta = 1.46e-09
#6, delta = 7.62e-12
#7, delta = 4.57e-14
#8, delta = 7.77e-16


In [13]:
# Her kan du spille av resultatet hvor vi har brukt Negentropy
ipd.display(ipd.Audio(sepData2[0,:], rate=sampling_rate))
ipd.display(ipd.Audio(sepData2[1,:], rate=sampling_rate))
ipd.display(ipd.Audio(sepData2[2,:], rate=sampling_rate))

### Resultat kun ved bruk av $\tilde{\mathbf{x}}$

In [14]:
'''
Under kan man allerede se effekten av bare de to første preprosesseringsstegene.
Her kan du høre de tre lydmiksene fra tilde{x} uten å ha blitt transformert av W
'''
ipd.display(ipd.Audio(Z1[0,:], rate=sampling_rate))
ipd.display(ipd.Audio(Z1[1,:], rate=sampling_rate))
ipd.display(ipd.Audio(Z1[2,:], rate=sampling_rate))

## Egne lydfiler
Nå skal vi prøve det samme med 4 lydfiler som er nedlastet fra nettet, og se at det også fungerer. Først mikser vi lydfilene, slik at vi får 4 forskjellige vektet lydmikser $\mathbf{x}_\mathrm{egen}$.

In [15]:
"""KODE HENTET FRA MAL, og modifisert fra 3 til 4 rader"""
def normalize_rowsums(A):
    the_sum = np.sum(A,1)
    A = A / the_sum.reshape((4,1))
    return A

def random_mixing_matrix(signals, observations):
    A = 0.25 + np.random.rand(observations, signals)
    return normalize_rowsums(A)

In [16]:
"""I denne cella laster jeg opp 4 lydklipp"""

c_paths = ['audio/custom_mix_1.wav', 'audio/custom_mix_2.wav', 'audio/custom_mix_3.wav', 'audio/custom_mix_4.wav']

c_data, c_sampling_rate = read_wavefiles(c_paths)
c_num_signals = c_data.shape[0]

In [17]:
"""Her starter vi miksingen"""
A = random_mixing_matrix(c_num_signals, c_num_signals)
data_mixed = normalize_audio(A @ c_data, c_num_signals)

In [18]:
"""Her får du de 4 resulterende miksene"""
ipd.display(ipd.Audio(data_mixed[0,:], rate=sampling_rate))
ipd.display(ipd.Audio(data_mixed[1,:], rate=sampling_rate))
ipd.display(ipd.Audio(data_mixed[2,:], rate=sampling_rate))
ipd.display(ipd.Audio(data_mixed[3,:], rate=sampling_rate))

### Kurtose (egne lydfiler)

In [19]:
cZ1, cW1 = fast_ICA(data_mixed, c_num_signals, kurtosis, tol)
cy1 = cW1 @ cZ1 # Kurtose

csepData1 = normalize_audio(cy1, c_num_signals) # gir lydkildene omtrent samme lydstyrke

#1, delta = 4.52e-01
#2, delta = 4.45e-02
#3, delta = 6.89e-03
#4, delta = 5.43e-03
#5, delta = 1.90e-03
#6, delta = 3.32e-04
#7, delta = 4.51e-05
#8, delta = 5.81e-06
#9, delta = 7.44e-07
#10, delta = 9.57e-08
#11, delta = 1.24e-08
#12, delta = 1.61e-09
#13, delta = 2.11e-10
#14, delta = 2.76e-11
#15, delta = 3.62e-12
#16, delta = 4.76e-13
#17, delta = 6.20e-14
#18, delta = 8.22e-15
#19, delta = 4.44e-16


In [20]:
"""Her kan du spille av resultatet"""
ipd.display(ipd.Audio(csepData1[0,:], rate=sampling_rate))
ipd.display(ipd.Audio(csepData1[1,:], rate=sampling_rate))
ipd.display(ipd.Audio(csepData1[2,:], rate=sampling_rate))
ipd.display(ipd.Audio(csepData1[3,:], rate=sampling_rate))

### Negentropy (egne lydfiler)

In [21]:
cZ2, cW2 = fast_ICA(data_mixed, c_num_signals, negentropy, tol)
cy2 = cW2 @ cZ2 # Negentropy

csepData2 = normalize_audio(cy2, c_num_signals) # gir lydkildene omtrent samme lydstyrke

#1, delta = 5.63e-01
#2, delta = 3.77e-02
#3, delta = 4.32e-02
#4, delta = 1.20e-02
#5, delta = 1.16e-02
#6, delta = 4.98e-03
#7, delta = 1.07e-03
#8, delta = 1.83e-04
#9, delta = 2.99e-05
#10, delta = 4.82e-06
#11, delta = 7.77e-07
#12, delta = 1.25e-07
#13, delta = 2.02e-08
#14, delta = 3.25e-09
#15, delta = 5.23e-10
#16, delta = 8.41e-11
#17, delta = 1.35e-11
#18, delta = 2.18e-12
#19, delta = 3.52e-13
#20, delta = 5.71e-14
#21, delta = 8.88e-15
#22, delta = 1.11e-15
#23, delta = 1.44e-15
#24, delta = 1.55e-15
#25, delta = 4.44e-16


In [22]:
"""Her kan du spille av resultatet"""
ipd.display(ipd.Audio(csepData1[0,:], rate=sampling_rate))
ipd.display(ipd.Audio(csepData1[1,:], rate=sampling_rate))
ipd.display(ipd.Audio(csepData1[2,:], rate=sampling_rate))
ipd.display(ipd.Audio(csepData1[3,:], rate=sampling_rate))

## Konklusjon og oppsummering
Vi får svært gode resultater med algoritmen. Den fungerer utmerket både med Kurtose, og med Egentropy, så jeg klarer ikke å skille noen forskjeller mellom de. Med de medfulgte lydfilene til prosjektet er det vanskelig å høre noe avvik, men det er mulig å høre litt bakgrunnslyder fra de andre kildene når man spiller av. Koden er blitt testet gjentatte ganger der vår tilfeldig genererte $\mathbf{W}_0$ hvor
$$
    \mathbf{W}_0 = \{\mathbf{W}_0 \in \mathbb{R}^{d\times d} : 0 \leq \mathbf{W}_{0,ij} < 1, \quad \forall \, i, j \in \{1, 2, \dots, d \}\},
$$
konvergerer et sted mellom 6 og 30 iterasjoner. Jeg har ikke opplevd at $\mathbf{W}_0$ ikke konvergerer en eneste gang, så etter minst 30 ganger med testing kan vi anta at algoritmen fungerer over 90 % av gangene programmet kjøres.

På de egneimporterte lydfilene som er hentet fra nettsiden [Freesound](https://freesound.org) får vi også svært gode resultat. Igjen kan man høre litt bakgrunnsstøy fra de andre kildene, men det høres ikke ut som det er noe tap på lydkvalitet. Her ser det ut til at lydfilene konvergererer etter gjennomsnittelig litt flere iterasjoner enn lydfilene som fulgte med prosjektet. Her konvergerer lydfilene alt ifra 7 til 80 iterasjoner. Det gir mening ettersom det er $4 \cdot 51\, 632 = 206\, 528$ elementer i `c_data` i stedet for $3 \cdot 50\, 000 = 150\, 000$ elementer i `data`. Etter ca. 15 ganger med testing, så har jeg opplevd at én gang at jeg fikk samme resultat i spor 3, og spor 4, at de var 50:50 mikset begge to. Dette gjaldt for både Kurtose og Egentropy, så én antagelse er at `random_mixing_matrix()` tilfeldigvis mikset `c_data` til en lineært avhengig matrise. Uansett er suksessraten sålangt svært høy, på ca. $14/15 \approx 93 \%$.

Algoritmen ble implementert iht. malen og formlene i oppgavebeskrivelsen, og testene er kommentert ut i cellene hvor funksjonene er implementert, så dette kan lett kontrolleres. En interessant oppdagelse var nemlig første gang jeg kjørte gjennom hele koden. Da glemte jeg å gjøre siste transformasjon, altså i stedet for å spille av fra
$$
    \mathbf{y} = \mathbf{W}\cdot\tilde{\mathbf{x}},
$$
så spilte jeg av direkte fra $\tilde{\mathbf{x}}$, og det kan man prøve ut selv lenger oppe i koden. Det var selvsagt interessant å se at koden "funket", men hadde håp om litt bedre resultat. Dette fikk jeg heldigvis etter jeg oppdaget feilen.